In [16]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta

In [3]:
df = pd.read_csv('final_df.csv')

In [4]:
df['Fetch_Time'] = pd.to_datetime(df['Fetch_Time'])

In [72]:
strike = df.Strike_Price.unique()

In [57]:
new = df[(df['Strike_Price'] == 24550) & (df['Expiry_Date']=='2025-05-08')]

In [58]:
new['Date'] = new['Fetch_Time'].dt.date

/tmp/ipykernel_716/788553029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Date'] = new['Fetch_Time'].dt.date


In [59]:
chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]

In [60]:
new[new['Date'] == datetime.date.today()]

,Fetch_Time,Symbol,Expiry_Date,CALLS_LTP,Strike_Price,PUTS_LTP,Date
305522,2025-05-02 09:17:14,NIFTY,2025-05-08,158.30,24550,251.70,2025-05-02
306357,2025-05-02 09:18:14,NIFTY,2025-05-08,158.10,24550,251.50,2025-05-02
307192,2025-05-02 09:19:14,NIFTY,2025-05-08,147.65,24550,248.50,2025-05-02
308029,2025-05-02 09:20:44,NIFTY,2025-05-08,144.50,24550,254.05,2025-05-02
308867,2025-05-02 09:21:44,NIFTY,2025-05-08,139.20,24550,249.90,2025-05-02
...,...,...,...,...,...,...,...
388157,2025-05-02 11:03:47,NIFTY,2025-05-08,136.15,24550,228.80,2025-05-02
389012,2025-05-02 11:04:47,NIFTY,2025-05-08,131.05,24550,238.00,2025-05-02
389867,2025-05-02 11:06:17,NIFTY,2025-05-08,137.15,24550,231.35,2025-05-02
390722,2025-05-02 11:06:47,NIFTY,2025-05-08,139.85,24550,224.30,2025-05-02


In [61]:
chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()

In [62]:

chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']


In [63]:
chk1['smma'] = Analysis.calculate_smma(chk1, period=7)

In [64]:
chk1['rsi'] = ta.rsi(chk1['Close'])

In [65]:
chk1['dema'] = ta.dema(chk1['Close'], length=10)

In [66]:
chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']

In [67]:
chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']

In [68]:
chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
 # np.where(chk1['changeover'] == 1 & chk1['condition'] == 1)

In [69]:
chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 

In [70]:
chk1

,datetime,Open,Low,High,Close,smma,rsi,dema,super_trend,super_trend_color,BBM,changeover,condition
